图像主要通过两种方式提供给模型：通过传递图像链接或直接在请求中传递 base64 编码的图像。图像可以在用户消息中传递。

该模型最擅长回答关于图像中存在内容的常见问题。尽管它确实理解图像中物体之间的关系，但它尚未优化以回答关于图像中某些物体位置的详细问题。例如，你可以询问它一辆车的颜色，或者根据冰箱里的食材询问晚餐的建议，但如果你展示一张房间的图片并询问椅子的位置，它可能无法正确回答这个问题。

# 通过链接读取图片

In [8]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "这张图片里有什么？"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这张图片展示了一条木质小路，蜿蜒穿过一片绿色的草地。草地上的绿草生机勃勃，背景有一些树木和灌木，天空中有蓝色的云朵，给人一种宁静自然的感觉。整体场景非常开阔，适合进行徒步旅行或野外观察。', refusal=None, role='assistant', function_call=None, tool_calls=None))


# Base64 方式

如果你有本地的图像或一组图像，你可以将它们以 base64 编码格式传递给模型，以下是一个实际操作的示例：

In [2]:
import base64
import requests


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "./archive/图片理解示例图.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "这张图片里有什么？"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这张图片展示了一条木栈道，延伸穿过一片绿色的草地和灌木丛，背景是蓝天和白云。整体景象显得宁静而自然，给人一种开阔的感觉。', refusal=None, role='assistant', function_call=None, tool_calls=None))


# 多张图片输入

Chat Completions API 能够接收并处理多个图像输入，这些图像可以是 base64 编码格式或作为图像 URL。模型将对每个图像进行处理，并利用所有图像的信息来回答问题。

In [4]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "这些图像里有什么？它们之间有什么区别吗？",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)
print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这两张图像是完全相同的。它们展示了一条木板路，漫步在绿色的草地中，背景是蓝天和白云。整体感觉宁静，展现了自然景观。\n\n因为它们是一模一样的，所以没有明显的区别。', refusal=None, role='assistant', function_call=None, tool_calls=None))


在这里，模型会看到两张相同的图像，并能够独立回答关于这两张图像或每张图像的问题。

# 低或高保真图像理解

通过控制细节参数，该参数有三个选项：低、高或自动，您可以控制模型如何处理图像并生成其文本理解。默认情况下，模型将使用自动设置，该设置会根据图像输入大小决定是否应使用低或高设置。
- `low` 将启用“低分辨率”模式。模型将接收一张低分辨率的 512px x 512px 图像版本，并以 85 个令牌的预算来表示该图像。这使得 API 能够更快地返回响应，并减少输入令牌的消耗，适用于不需要高细节的用例。
- `high` 将首先允许模型查看低分辨率图像（使用 85 个令牌），然后为每个 512px x 512px 的图块创建详细裁剪，每个裁剪使用 170 个令牌。

In [7]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "这些图像里有什么？"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
            "detail": "high"
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

res = response
print(res)
print(res.choices[0].message.content)

ChatCompletion(id='chatcmpl-9wOHs987FtvOAISSn80aLZ3j2oxE6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图像展示了一条木栈道，穿过一片绿色的草地，背景是蓝天和一些白云。周围有树木和灌木，整体环境显得宁静而自然。', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723703412, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_507c9469a1', usage=CompletionUsage(completion_tokens=49, prompt_tokens=36848, total_tokens=36897))
这幅图像展示了一条木栈道，穿过一片绿色的草地，背景是蓝天和一些白云。周围有树木和灌木，整体环境显得宁静而自然。


在低分辨率模式下，我们期望图像尺寸为 512px x 512px。在高分辨率模式下，图像的短边应小于 768px，长边应小于 2,000px。

# 其他局限性
尽管具备视觉功能的 GPT-4 功能强大，并可在多种场景中应用，但了解其模型局限性至关重要。以下是我们已知的一些限制：
- 医学影像：该模型不适用于解读如 CT 扫描等专业医学影像，也不应用于提供医疗建议。
- 非英语：在处理包含非拉丁字母文本的图像时，例如中文，模型可能无法达到最佳性能。
- 小号文字：放大图片中的文字以提高可读性，但避免裁剪重要细节。
- 旋转：模型可能误解旋转或倒置的文字或图像。
- 视觉元素：模型可能在理解颜色或样式（如实线、虚线或点线）变化的图表或文本时遇到困难。
- 空间推理：模型在需要精确空间定位的任务上表现不佳，例如识别棋盘位置。
- 图像形状：该模型在处理全景图和鱼眼图像时存在困难。
- 元数据与调整大小：模型不处理原始文件名或元数据，图像在分析前会被调整大小，影响其原始尺寸。
- 计数：可对图像中的对象进行大致计数。
- 验证码：出于安全考虑，我们已实施系统来阻止验证码的提交。
- 我们目前支持 PNG (.png)、JPEG (.jpeg 和 .jpg)、WEBP (.webp) 以及非动画 GIF (.gif)。
- 每张图片的上传限制为 20MB。